In [4]:
#check ieach column if NaN
def checkColIfNaN(dff):
    for i in dff.columns:
        if dff[i].isna.empty:
            pass
            

In [8]:

import re

def detect_datetime_columns(df):
    result = {}

    for col in df.columns:
        col_data = df[col]

        # Handle if column is numeric and looks like a year
        if pd.api.types.is_integer_dtype(col_data) or pd.api.types.is_float_dtype(col_data):
            if col_data.dropna().empty == False:
                if col_data.dropna().between(1800, 2100).all():
                    result[col] = 1  # Only year
                    continue

        # Convert to string for flexible matching
        col_str = df[col].astype(str).str.strip()

        # Handle year-like strings
        if col_str.str.match(r'^\d{4}$').all():
            print("date is str")
            if col_str.astype(int).between(1800, 2100).all():
                result[col] = 1  # Only year
                continue

        # Check if values have full date (yyyy-mm-dd, dd-mm-yyyy, etc.)
        if col_str.str.match(r'^\d{1,4}[-/\.]\d{1,2}[-/\.]\d{1,4}$').all():
            result[col] = 2  # Only date
            continue

        # Check if values have date AND time
        if col_str.str.contains(r'\d{1,4}[-/\.]\d{1,2}[-/\.]\d{1,4}.*\d{1,2}:\d{2}').all():
            result[col] = 3  # Date + Time
            continue

        # Else not a recognized datetime pattern
        result[col] = 0

    return result


In [36]:
import pandas as pd
import numpy as np
from pandas.api.types import is_datetime64_any_dtype

mainDf = pd.read_csv('customers-1000.csv')#df is the data frame containing the whole content of file
columns = np.array(mainDf.columns)#to know the number of colums in dataframe(csv) file
noOfColumns = len(columns)
noOfRows = len(mainDf.index)
print(f"Total columns = {noOfColumns}")
print(f"Total rows = {noOfRows}")
sortAccordingToDateTime = detect_datetime_columns(mainDf)
print(sortAccordingToDateTime)
for col in mainDf.columns:
    if sortAccordingToDateTime[col] in [1,2,3]:
        mainDf = clusterDateTimeCol(mainDf,col,sortAccordingToDateTime[col]) 
        arrOfGroupNames = mainDf.index.get_level_values(0).unique()# this variable i containing the group names
    else:
        continue
print(mainDf)


Total columns = 12
Total rows = 1000
{'Index': 0, 'Customer Id': 0, 'First Name': 0, 'Last Name': 0, 'Company': 0, 'City': 0, 'Country': 0, 'Phone 1': 0, 'Phone 2': 0, 'Email': 0, 'Subscription Date': 2, 'Website': 0}
Subscription Date date called 
last index called
{2020: 425, 2021: 829, 2022: 999}
[426, 404, 170]
Multicalled
Date Or DateTime
                           Index      Customer Id First Name Last Name  \
Group              Sr No.                                                
Group of year 2020 0          40  BEBA4fDAA6C4adC     Rickey      Mays   
                   1         242  dEc837d5F13C1ed     Kendra    Waters   
                   2         617  F9AEc2F51C5EDaE     Parker     Russo   
                   3         148  EF5858dEe5f7649    Belinda  Ferguson   
                   4         952  a9feB7dCD2FFd4e      Larry       Key   
...                          ...              ...        ...       ...   
Group of year 2022 995       772  cDBe5cFdbB3Ae4F     Jasmin  

C:\Users\tikes\AppData\Local\Temp\ipykernel_2816\330670565.py:17: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  parsed_col = pd.to_datetime(fContent[col], errors='raise')
C:\Users\tikes\AppData\Local\Temp\ipykernel_2816\4054971873.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_indices = df.groupby('year').apply(lambda x: x.index[-1]).to_dict()


In [ ]:
def clusterDateTimeCol(fContent, col,no):
    if no ==1:
        # Try to detect and sort if the column is just year values
        fContent = fContent.sort_values(by=col,ignore_index=True, ascending=True)
        fContent = fContent.reset_index(drop=True)
        fContent = multiIndex(fContent,col, True)
    elif no == 2 or 3:
        # Now, try to detect proper date/datetime columns
        try:
            # Avoid processing numeric-only or zero-filled columns
          
            sample_vals = fContent[col].astype(str).str.strip().replace('0', np.nan).dropna()
            if len(sample_vals) == 0:
                return fContent# All values are zero or empty-like
            # Try parsing
            try:
                parsed_col = pd.to_datetime(fContent[col], errors='raise')
            except:
                parsed_col = pd.to_datetime(fContent[col], dayfirst=True, errors='raise')
            if all(parsed_col.dt.time == pd.to_datetime('00:00:00').time()) and no == 2:  # Only date
                fContent[col] = parsed_col
                fContent = clean_and_sort_date_column(fContent, col, True)
                fContent = fContent.reset_index(drop=True)
                fContent = multiIndex(fContent,col)
                # Replace original column with parsed datetime values
            elif no == 3:  # Date + time
                fContent[col] = parsed_col
                fContent = handle_datetime_column(fContent, col, False)
                fContent = fContent.reset_index(drop=True)
                fContent = multiIndex(fContent,col)
            print("Date Or DateTime")
        except Exception as e:
            print("date year  datetime col flop")
            return fContent # Not a datetime column
        
   
    return fContent 
    


In [3]:
#fuction which will sort the dates if the df has date containing columns
def clean_and_sort_date_column(dff, column_name, ascending=True):
        try:
            # Step 1: Convert to datetime (handles multiple formats)
            dff[column_name] = pd.to_datetime(dff[column_name], errors='coerce', dayfirst=True)
            
            # Step 2: Drop NaT (invalid formats)
            dff = dff.dropna(subset=[column_name])
            
            # Step 3: Sort the DataFrame by that column
            dff = dff.sort_values(by=column_name, ascending=ascending)

            # Optional: Format to clean date string (YYYY-MM-DD)
            dff[column_name] = dff[column_name].dt.strftime('%Y-%m-%d')
            print(f"{column_name} date called ")
            return dff
        
        except Exception as e:
            print(f"⚠️ Error while processing date column: {e}")
            return dff

In [4]:
# if df has column containing the date and time both
def handle_datetime_column(df, column_name, ascending):
    print(f"{column_name} dateTime")
    # Check if most values in column are datetime with time
    values = df[column_name].dropna().astype(str).head(20)
    count_datetime = sum([is_datetime(v) for v in values])

    if count_datetime >= len(values) // 2:  # At least half must be datetime-like
        # Convert full column to datetime
        df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
        # Drop rows with invalid dates
        df = df.dropna(subset=[column_name])
        # Sort by that column
        df = df.sort_values(by=column_name,ascending=ascending).reset_index(drop=True)
        print(f"[INFO] '{column_name}' successfully recognized and sorted as datetime.")
    else:
        print(f"[INFO] '{column_name}' does not contain proper datetime with time.")

    return df

In [5]:

#Fuction to get the years from teh date format
def extract_year(date_str):
    match = re.search(r'\b(19|20)\d{2}\b', date_str)
    if match:
        return int(match.group(0))
    return None

# Function to extract all unique years from a pandas Series
def extract_all_years(series):
    print("extract_all_years called")
    setofyears = [None] * len(series)

    for i in range(len(series)):
        setofyears[i] = extract_year(str(series[i]).strip())

    # Filter None values and return sorted unique years
    return np.array(sorted(set(y for y in setofyears if y is not None)))


def get_last_indices_of_each_year(date_series, YearOnly=False):
    
    # data_series = data_series.apply(pd.to_numeric, errors='coerce').astype('Int64')
    if YearOnly == True:
        print("yearOnly work")
        df = pd.DataFrame({'year': date_series}, index=np.arange(len(date_series)))
    
    else:
        # Extract year
        years = date_series.dt.year
        
        # Create a DataFrame with index
        df = pd.DataFrame({'year': years}, index=np.arange(len(date_series)))
        
    # Get last index of each year group
    last_indices = df.groupby('year').apply(lambda x: x.index[-1]).to_dict()
    print("last index called")
    return last_indices


In [6]:
#Group(MultiIndexing) accordind to the condition 
def multiIndex(dataFrame, colToCheck, yearOnly = False):
    # Step 1: Get last indices of each year
    if yearOnly == True:
        yearsWithLastIndex = get_last_indices_of_each_year(dataFrame[colToCheck], True)
    else:
        yearsWithLastIndex = get_last_indices_of_each_year(pd.to_datetime(dataFrame[colToCheck]))
    print(yearsWithLastIndex)
    nameOfGroups = list(yearsWithLastIndex.keys())
    last_indices = list(yearsWithLastIndex.values())

    # Step 2: Compute counts from last indices
    group_sizes = []
    prev = -1
    for idx in last_indices:
        group_sizes.append(idx - prev)
        prev = idx
    print(group_sizes)
    # Step 3: Create array per group
    objOfGroups = {
        f'key{i}': np.array([f'Group of year {year}'] * group_sizes[i])
        for i, year in enumerate(nameOfGroups)
    }
    
    # Step 4: Combine into one array
    outside = np.concatenate(list(objOfGroups.values()))
    
    # Step 5: Create inside index
    inside = np.arange(len(outside))
    
    multi_index = pd.MultiIndex.from_arrays([outside, inside], names=["Group", "Sr No."])
    
    dataFrame = dataFrame.reindex(range(len(multi_index)))
    dataFrame.set_index(multi_index,inplace=True,)
    print("Multicalled")
    return dataFrame


In [7]:
# Now time to cluster the file(data)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
@app.route("/clusteringStart", methods=["POST"])
def dataClustering():#FileToCluster is DF which is created to print through html table
    #Process of creating appropriate DF to apply KMeans
    noOfClusters = int(request.form.get("noOfCluster"))
    fileToClusterNumaric = MainDF.select_dtypes(include=["float64", "int64"])# to Change the string type columns to int or float because KMeans() needs int or float type colum to mesure distance
    scaler = StandardScaler()
    scaled_data = (scaler.fit_transform((fileToClusterNumaric)))
    
    # Process of applying kMeans
    Kmeans = KMeans(n_clusters = noOfClusters, random_state=42)
    Kmeans_lables = Kmeans.fit(scaled_data)
    # hence means returns a array So need to change into Dataframe
    clusteredDF = pd.DataFrame(Kmeans_lables)

    return render_template("index.html", clusteredData = clusteredDF.to_html(classes="UploadedclusteredData", border=0))

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
#if Only year
def sort_if_year_column(column):
    print(f"{column.name } year only call")
    try:
        # Step 1: Convert to string and strip whitespace
        cleaned = column.astype(str).str.strip()
        
        # Step 2: Check if all values match 4-digit pattern
        if cleaned.str.fullmatch(r'\d{4}').all():
            # Step 3: Convert to int and check valid year range
            years = cleaned.astype(int)
            if years.between(1800, 2050).all():
                print("✅ Year-only column detected. Sorted.")
                return years.sort_values(ignore_index=True)
            else:
               pass
        else:
            pass
    except Exception as e:
        print(f"Error: {e}")
    
    return False # Return as-is if not valid


In [47]:
obj = {
    'o':0,
    'oo':0,
    'ooo':2,
    'oooo':1
}
arr  = obj.values()
if 1 in  arr or 2 in arr or 3 in arr:
    print("yes")
else:
    print("no")

yes
